In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, LeakyReLU
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time


In [2]:
EPOCH_COUNT = 50

NAME = f"single_rgb_image_regression_V02_epochs_{EPOCH_COUNT}_{int(time.time())}"


In [3]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
# def create_data_via_ImageDataGenerator():
#     train = ImageDataGenerator(rescale=1/255)
#     validation = ImageDataGenerator(rescale=1/255)
#     train_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                               target_size=(60, 60), batch_size=3, class_mode='binary')
#     validation_dataset = train.flow_from_directory("../Data/Train/smallTown",
#                                                target_size=(60, 60), batch_size=3, class_mode='binary')


In [5]:
def load_data_via_numpy_load():
    return (np.load("CNN_trainset_single_RGB_regression_V02_X.npy")/255, np.load("CNN_trainset_single_RGB_regression_V02_Y.npy"))


In [6]:
X, Y = load_data_via_numpy_load()


In [7]:
# X



In [8]:
X.shape

(60000, 60, 60, 3)

In [9]:


X = X.reshape(-1, 60, 60, 3)

X.shape


(60000, 60, 60, 3)

In [10]:
def build_model():
    model = Sequential()

    # 1st Conv-Layer
    model.add(Conv2D(128, (3, 3), input_shape=(X.shape[1:])))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))

    # 2nd Conv-Layer
    model.add(Conv2D(192, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    # 3rd Conv-Layer
    model.add(Conv2D(320, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    # 4th Conv-Layer
    model.add(Conv2D(576, (3, 3)))
    model.add(Activation(LeakyReLU(alpha=0.2)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())

    # 1st Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 2nd Fully-Connected
    model.add(Dense(4096))
    model.add(Activation(LeakyReLU(alpha=0.2)))

    # 3rd Fully-Connected
    model.add(Dense(1))
    # model.add(Activation("sigmoid"))
    return model


model = build_model()


In [11]:

model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 128)       3584      
_________________________________________________________________
activation (Activation)      (None, 58, 58, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 58, 58, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 192)       221376    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 192)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 192)       7

In [12]:
model.fit(X, Y, batch_size=32, epochs=EPOCH_COUNT, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/50
1688/1688 [==============================] - 123s 52ms/step - loss: 1.2097 - accuracy: 0.3803 - val_loss: 0.0219 - val_accuracy: 0.3990
Epoch 2/50
1688/1688 [==============================] - 80s 48ms/step - loss: 0.0179 - accuracy: 0.3933 - val_loss: 0.0139 - val_accuracy: 0.3970
Epoch 3/50
1688/1688 [==============================] - 80s 47ms/step - loss: 0.0132 - accuracy: 0.3943 - val_loss: 0.0094 - val_accuracy: 0.3992
Epoch 4/50
1688/1688 [==============================] - 80s 47ms/step - loss: 23.0746 - accuracy: 0.3860 - val_loss: 39.8869 - val_accuracy: 0.1897
Epoch 5/50
1688/1688 [==============================] - 80s 47ms/step - loss: 0.1733 - accuracy: 0.3531 - val_loss: 0.3343 - val_accuracy: 0.3355
Epoch 6/50
1688/1688 [==============================] - 83s 49ms/step - loss: 0.0863 - accuracy: 0.3716 - val_loss: 0.0383 - val_accuracy: 0.3983
Epoch 7/50
1688/1688 [==============================] - 82s 49ms/step - loss: 0.0594 - accuracy: 0.3797 - val_loss: 0.037

# Test


In [13]:
# model = tf.keras.models.load_model("./models/single_rgb_image_big_dataset")


In [14]:
# # Alias

# img1 = Image.open(
#     "D:\Main\MA_PROGR\Data\Train\LED_Wand_Aufnahmen\Alias\LED_Wand_20001.png")
# img1 = np.asarray(img1)/255




# img1 = img1.reshape(1,60,60,3)
# print(img1.shape)
# # print(img1)
# res1 = model.predict(img1)
# print(res1)


In [15]:
# # NoAlias

# img2 = Image.open(
#     "D:\\Main\\MA_PROGR\\Data\\Train\\LED_Wand_Aufnahmen\\NoAlias\\abstract_cross_203.png")
# img2 = np.asarray(img2)/255

# img2 = img2.reshape(1, 60, 60, 3)
# img2.shape
# # print(img2)
# res2 = model.predict(img2)
# print(res2)


In [16]:
# import matplotlib.pyplot as plt
# plt.imshow(img2.reshape(60, 60, 3))


In [17]:
tf.keras.models.save_model(model, "./models/{}".format(NAME))

INFO:tensorflow:Assets written to: ./models/single_rgb_image_regression_V02_epochs_50_1653373797\assets


INFO:tensorflow:Assets written to: ./models/single_rgb_image_regression_V02_epochs_50_1653373797\assets
